## Long Short-Term Memory (LSTM) Networks

**1. Literature:**

* **Motivation:**
    * Recurrent Neural Networks (RNNs) are designed to process sequential data, but they struggle with long-range dependencies due to the vanishing/exploding gradient problem.
    * LSTMs, introduced by Hochreiter and Schmidhuber in 1997, address this by introducing a memory cell and gating mechanisms.
* **Key Idea:**
    * LSTMs use "gates" to control the flow of information, allowing them to selectively remember or forget past information.
    * The "cell state" acts as a conveyor belt, carrying information through the sequence.
* **Applications:**
    * Natural language processing (NLP), speech recognition, time series analysis, and more.

**2. Mathematics:**

* **LSTM Cell:**
    * An LSTM cell maintains a cell state ($C_t$) and hidden state ($h_t$).
    * It uses three gates: forget gate ($f_t$), input gate ($i_t$), and output gate ($o_t$).
    * These gates are computed using sigmoid functions, which produce values between 0 and 1.
    * The core equations are:

    * **Forget Gate:**
        * $f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$
    * **Input Gate:**
        * $i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$
        * $\tilde{C}_t = \tanh(W_c \cdot [h_{t-1}, x_t] + b_c)$
    * **Cell State Update:**
        * $C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$
    * **Output Gate:**
        * $o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$
    * **Hidden State:**
        * $h_t = o_t \odot \tanh(C_t)$
    * Where:
        * $\sigma$ is the sigmoid function.
        * $\tanh$ is the hyperbolic tangent function.
        * $\odot$ is element-wise multiplication.
        * $W$ are weight matrices.
        * $b$ are bias vectors.
        * $x_t$ is the input at time step $t$.

**3. Graphics:**

+---------------+
|   LSTM Cell   |
+---------------+
|
v
+-----------------+
| Forget Gate (ft) |
+-----------------+
|
v
+-----------------+
| Input Gate (it)  |
+-----------------+
|
v
+-----------------+
| Output Gate (ot) |
+-----------------+
|
v
+-----------------+
| Cell State (Ct)  |
+-----------------+
|
v
+-----------------+
| Hidden State (ht)|
+-----------------+


Visualizing the LSTM cell:

      +-----------+
      |  Sigmoid  |
      +-----------+
      |           |
x_t ----> |  Input    |
|  Gate (it)|
+-----------+
|           |
h_t-1 --> |           |
v           v
+-----------+    +-----------+
|  Sigmoid  |    |  Tanh     |
+-----------+    +-----------+
|           |    |           |
|  Forget   |    |  Candidate|
|  Gate (ft)|    |  Cell (Ct~) |
+-----------+    +-----------+
|           |
v           v
+-------+     +-------+
|  * |     |  * |
+-------+     +-------+
|           |
+-----+-----+
|
v
+-----------+
|  Cell     |
|  State (Ct)|
+-----------+
|
v
+-----------+
|  Tanh     |
+-----------+
|
v
+-----------+
|  Sigmoid  |
+-----------+
|           |
|  Output   |
|  Gate (ot)|
+-----------+
|
v
+-------+
|  * |
+-------+
|
v
h_t



In [1]:
import numpy as np

In [2]:
class LSTM:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases
        self.Wf = np.random.randn(hidden_size, input_size + hidden_size) / np.sqrt(input_size + hidden_size)
        self.Wi = np.random.randn(hidden_size, input_size + hidden_size) / np.sqrt(input_size + hidden_size)
        self.Wc = np.random.randn(hidden_size, input_size + hidden_size) / np.sqrt(input_size + hidden_size)
        self.Wo = np.random.randn(hidden_size, input_size + hidden_size) / np.sqrt(input_size + hidden_size)
        self.Wy = np.random.randn(output_size, hidden_size) / np.sqrt(hidden_size)

        self.bf = np.zeros((hidden_size, 1))
        self.bi = np.zeros((hidden_size, 1))
        self.bc = np.zeros((hidden_size, 1))
        self.bo = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def forward(self, inputs):
        T = len(inputs)
        h = np.zeros((T + 1, self.hidden_size, 1))
        c = np.zeros((T + 1, self.hidden_size, 1))
        o = np.zeros((T, self.output_size, 1))

        for t in range(T):
            x = np.array(inputs[t]).reshape(-1, 1)
            z = np.vstack((h[t], x))

            ft = self.sigmoid(np.dot(self.Wf, z) + self.bf)
            it = self.sigmoid(np.dot(self.Wi, z) + self.bi)
            ct_tilde = self.tanh(np.dot(self.Wc, z) + self.bc)
            c[t + 1] = ft * c[t] + it * ct_tilde
            ot = self.sigmoid(np.dot(self.Wo, z) + self.bo)
            h[t + 1] = ot * self.tanh(c[t + 1])
            o[t] = np.dot(self.Wy, h[t + 1]) + self.by

        return o

In [3]:
# Example Usage
input_size = 3
hidden_size = 4
output_size = 2
lstm = LSTM(input_size, hidden_size, output_size)
inputs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
outputs = lstm.forward(inputs)
print(outputs)

[[[ 0.18643346]
  [-0.20957958]]

 [[ 0.41041454]
  [-0.44130399]]

 [[ 0.50196474]
  [-0.57346038]]]
